# VQ-VAE Embedding Validation V3 - Comprehensive Diagnostics

## Goals

1. **Determine which embedding is best** for VQ-VAE structure reconstruction
2. **Understand WHY** one embedding outperforms another
3. **Identify failure modes** - what's breaking and how to fix it
4. **Guide future improvements** - what aspects of embeddings/VQ-VAE matter most

## Embeddings Tested

| Embedding | Dim | Description |
|-----------|-----|-------------|
| V1 | 32 | Skip-gram (co-occurrence based) |
| V3 | 40 | Compositional (material + shape + properties) |
| Random | 32 | Random baseline |

**Note:** V2 (hybrid) dropped - consistently worst performer in previous runs.

## Configuration

- **Epochs**: 12
- **Seeds**: 1 (single run per embedding, ~5 hours total)
- **Structure weight**: 10x (to counter 80% air imbalance)

## Improvements over V2 Validation

| Category | V2 Validation | V3 (this notebook) |
|----------|---------------|--------------------|
| **VQ-VAE** | Broken config | **EMA codebook, dead code reset** |
| **Per-block analysis** | None | **Top/bottom blocks, confusion matrix** |
| **Latent space** | Usage only | **Perplexity, entropy, code analysis** |
| **Visual** | None | **Reconstruction samples** |
| **Training dynamics** | Basic | **Gradient norms, loss components** |
| **Block-type breakdown** | None | **Accuracy by block category** |

## Key Questions to Answer

1. Does V1 or V3 perform better?
2. Which block types benefit most from good embeddings?
3. What do the codebook entries represent?
4. Are structures visually recognizable after reconstruction?
5. What's the main failure mode - wrong block type, or wrong position?

In [ ]:
# ============================================================
# CELL 1: Imports
# ============================================================

import json
import random
import time
from pathlib import Path
from typing import Dict, List, Tuple, Any, Optional
from collections import defaultdict, Counter

import h5py
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from scipy import stats

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# ============================================================
# CELL 2: Configuration
# ============================================================

# === Data Paths ===
DATA_DIR = "/kaggle/input/minecraft-schematics/minecraft_splits/splits/train"
VAL_DIR = "/kaggle/input/minecraft-schematics/minecraft_splits/splits/val"
VOCAB_PATH = "/kaggle/input/minecraft-schematics/tok2block.json"

# Embeddings (V2 dropped - consistently worst performer)
V1_EMBEDDINGS_PATH = "/kaggle/input/minecraft-embeddings/block_embeddings.npy"
V3_EMBEDDINGS_PATH = "/kaggle/input/minecraft-embeddings-v3/block_embeddings_v3.npy"

OUTPUT_DIR = "/kaggle/working"

# === Model Architecture ===
HIDDEN_DIMS = [64, 128, 256]
LATENT_DIM = 256
NUM_CODEBOOK_ENTRIES = 512

# === VQ-VAE Settings ===
COMMITMENT_COST = 0.5
EMA_DECAY = 0.99
DEAD_CODE_THRESHOLD = 2
STRUCTURE_WEIGHT = 10.0

# === Training ===
EPOCHS = 12
BATCH_SIZE = 4
LEARNING_RATE = 3e-4
USE_AMP = True
GRAD_ACCUM_STEPS = 4

# === Statistical Rigor ===
NUM_SEEDS = 1  # Single seed to fit within Kaggle time limits
BASE_SEEDS = [42]

# === Diagnostics ===
NUM_RECONSTRUCTION_SAMPLES = 5  # Visual samples to save
TOP_K_BLOCKS = 20  # Track top/bottom K blocks by accuracy

NUM_WORKERS = 2

print("V3 Validation Configuration:")
print(f"  Embeddings: V1, V3, Random (V2 dropped)")
print(f"  Seeds: {BASE_SEEDS} ({NUM_SEEDS} runs per embedding)")
print(f"  Epochs: {EPOCHS}")
print(f"  Architecture: {HIDDEN_DIMS}")
print(f"  Structure weight: {STRUCTURE_WEIGHT}x")

In [ ]:
# ============================================================
# CELL 3: Load Vocabulary and Categorize Blocks
# ============================================================

with open(VOCAB_PATH, 'r') as f:
    tok2block = {int(k): v for k, v in json.load(f).items()}

block2tok = {v: k for k, v in tok2block.items()}
VOCAB_SIZE = len(tok2block)
print(f"Vocabulary size: {VOCAB_SIZE}")

# Find air tokens
AIR_TOKENS = set()
for tok, block in tok2block.items():
    if 'air' in block.lower() and 'stair' not in block.lower():
        AIR_TOKENS.add(tok)
        print(f"  Air token: {tok} = {block}")

AIR_TOKENS_LIST = sorted(AIR_TOKENS)
AIR_TOKENS_TENSOR = torch.tensor(AIR_TOKENS_LIST, dtype=torch.long)

# Categorize blocks for analysis
BLOCK_CATEGORIES = {
    'wood': [], 'stone': [], 'metal': [], 'glass': [],
    'wool': [], 'concrete': [], 'terracotta': [],
    'stairs': [], 'slabs': [], 'walls': [], 'fences': [],
    'doors': [], 'plants': [], 'redstone': [], 'light': [],
    'other': []
}

for tok, block in tok2block.items():
    if tok in AIR_TOKENS:
        continue
    block_lower = block.lower()
    
    categorized = False
    if any(w in block_lower for w in ['oak', 'spruce', 'birch', 'jungle', 'acacia', 'dark_oak', 'mangrove', 'cherry', 'bamboo', 'crimson', 'warped']):
        if 'stair' in block_lower:
            BLOCK_CATEGORIES['stairs'].append(tok)
        elif 'slab' in block_lower:
            BLOCK_CATEGORIES['slabs'].append(tok)
        elif 'fence' in block_lower:
            BLOCK_CATEGORIES['fences'].append(tok)
        elif 'door' in block_lower:
            BLOCK_CATEGORIES['doors'].append(tok)
        else:
            BLOCK_CATEGORIES['wood'].append(tok)
        categorized = True
    elif any(w in block_lower for w in ['stone', 'cobble', 'brick', 'granite', 'diorite', 'andesite', 'deepslate', 'tuff']):
        if 'stair' in block_lower:
            BLOCK_CATEGORIES['stairs'].append(tok)
        elif 'slab' in block_lower:
            BLOCK_CATEGORIES['slabs'].append(tok)
        elif 'wall' in block_lower:
            BLOCK_CATEGORIES['walls'].append(tok)
        else:
            BLOCK_CATEGORIES['stone'].append(tok)
        categorized = True
    elif any(w in block_lower for w in ['iron', 'gold', 'copper', 'netherite']):
        BLOCK_CATEGORIES['metal'].append(tok)
        categorized = True
    elif 'glass' in block_lower:
        BLOCK_CATEGORIES['glass'].append(tok)
        categorized = True
    elif 'wool' in block_lower:
        BLOCK_CATEGORIES['wool'].append(tok)
        categorized = True
    elif 'concrete' in block_lower:
        BLOCK_CATEGORIES['concrete'].append(tok)
        categorized = True
    elif 'terracotta' in block_lower:
        BLOCK_CATEGORIES['terracotta'].append(tok)
        categorized = True
    elif any(w in block_lower for w in ['redstone', 'piston', 'observer', 'comparator', 'repeater', 'lever', 'button']):
        BLOCK_CATEGORIES['redstone'].append(tok)
        categorized = True
    elif any(w in block_lower for w in ['torch', 'lantern', 'lamp', 'glowstone', 'sea_lantern', 'shroomlight']):
        BLOCK_CATEGORIES['light'].append(tok)
        categorized = True
    elif any(w in block_lower for w in ['flower', 'grass', 'fern', 'leaves', 'sapling', 'vine', 'moss']):
        BLOCK_CATEGORIES['plants'].append(tok)
        categorized = True
    
    if not categorized:
        BLOCK_CATEGORIES['other'].append(tok)

print("\nBlock categories:")
for cat, toks in BLOCK_CATEGORIES.items():
    print(f"  {cat}: {len(toks)} blocks")

In [ ]:
# ============================================================
# CELL 4: Load Embeddings
# ============================================================

# V1: Skip-gram embeddings
v1_embeddings = np.load(V1_EMBEDDINGS_PATH).astype(np.float32)
print(f"V1: {v1_embeddings.shape}")

# V3: Compositional embeddings
v3_embeddings = np.load(V3_EMBEDDINGS_PATH).astype(np.float32)
print(f"V3: {v3_embeddings.shape}")

# Random (will create fresh per seed)
v1_std = v1_embeddings.std()
print(f"V1 std (for random init): {v1_std:.4f}")

# V2 dropped - consistently worst performer in previous validation
EMBEDDINGS_BASE = {
    'V1': {'embeddings': v1_embeddings, 'dim': 32},
    'V3': {'embeddings': v3_embeddings, 'dim': 40},
}

print(f"\nEmbeddings to test: {list(EMBEDDINGS_BASE.keys())} + Random")

In [ ]:
# ============================================================
# CELL 5: Dataset
# ============================================================

class VQVAEDataset(Dataset):
    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
        self.h5_files = sorted(self.data_dir.glob("*.h5"))
        if not self.h5_files:
            raise ValueError(f"No H5 files in {data_dir}")
        print(f"Found {len(self.h5_files)} structures in {data_dir}")
    
    def __len__(self):
        return len(self.h5_files)
    
    def __getitem__(self, idx):
        with h5py.File(self.h5_files[idx], 'r') as f:
            key = list(f.keys())[0]
            structure = f[key][:].astype(np.int64)
        return torch.from_numpy(structure).long(), idx

train_dataset = VQVAEDataset(DATA_DIR)
val_dataset = VQVAEDataset(VAL_DIR)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

In [ ]:
# ============================================================
# CELL 6: Enhanced VQ-VAE with Diagnostics
# ============================================================

class ResidualBlock3D(nn.Module):
    def __init__(self, channels: int):
        super().__init__()
        self.conv1 = nn.Conv3d(channels, channels, 3, padding=1)
        self.conv2 = nn.Conv3d(channels, channels, 3, padding=1)
        self.bn1 = nn.BatchNorm3d(channels)
        self.bn2 = nn.BatchNorm3d(channels)
    
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        return F.relu(x + residual)


class VectorQuantizerEMA(nn.Module):
    def __init__(self, num_codes, latent_dim, commitment_cost=0.5, ema_decay=0.99, dead_threshold=2):
        super().__init__()
        self.num_codes = num_codes
        self.latent_dim = latent_dim
        self.commitment_cost = commitment_cost
        self.ema_decay = ema_decay
        self.dead_threshold = dead_threshold
        
        self.register_buffer('codebook', torch.randn(num_codes, latent_dim))
        self.codebook.data.uniform_(-1/num_codes, 1/num_codes)
        self.register_buffer('ema_cluster_size', torch.zeros(num_codes))
        self.register_buffer('ema_embed_sum', torch.zeros(num_codes, latent_dim))
        self.register_buffer('code_usage', torch.zeros(num_codes))
        self.register_buffer('code_usage_total', torch.zeros(num_codes))
    
    def reset_epoch_stats(self):
        self.code_usage.zero_()
    
    def get_usage_fraction(self):
        return (self.code_usage > 0).float().mean().item()
    
    def get_perplexity(self):
        """Effective number of codes used (2^entropy)."""
        if self.code_usage.sum() == 0:
            return 0.0
        probs = self.code_usage / self.code_usage.sum()
        probs = probs[probs > 0]  # Avoid log(0)
        entropy = -(probs * probs.log()).sum()
        return entropy.exp().item()
    
    def get_entropy(self):
        """Entropy of code distribution (higher = more uniform)."""
        if self.code_usage.sum() == 0:
            return 0.0
        probs = self.code_usage / self.code_usage.sum()
        probs = probs[probs > 0]
        return -(probs * probs.log()).sum().item()
    
    def forward(self, z_e):
        z_e_perm = z_e.permute(0, 2, 3, 4, 1).contiguous()
        flat = z_e_perm.view(-1, self.latent_dim)
        
        # Cast to float32 for distance computation (avoid AMP half precision issues)
        flat_f32 = flat.float()
        codebook_f32 = self.codebook.float()
        
        d = (flat_f32.pow(2).sum(1, keepdim=True) 
             + codebook_f32.pow(2).sum(1) 
             - 2 * flat_f32 @ codebook_f32.t())
        
        indices = d.argmin(dim=1)
        
        # Track usage
        with torch.no_grad():
            for idx in indices.unique():
                count = (indices == idx).sum()
                self.code_usage[idx] += count
                self.code_usage_total[idx] += count
        
        z_q_flat = self.codebook[indices]
        z_q_perm = z_q_flat.view(z_e_perm.shape)
        
        if self.training:
            with torch.no_grad():
                encodings = F.one_hot(indices, self.num_codes).float()
                batch_size = encodings.sum(0)
                
                self.ema_cluster_size = self.ema_decay * self.ema_cluster_size + (1 - self.ema_decay) * batch_size
                batch_sum = encodings.t() @ flat_f32  # Use float32
                self.ema_embed_sum = self.ema_decay * self.ema_embed_sum + (1 - self.ema_decay) * batch_sum
                
                n = self.ema_cluster_size.sum()
                smoothed = (self.ema_cluster_size + 1e-5) / (n + self.num_codes * 1e-5) * n
                self.codebook.data = self.ema_embed_sum / smoothed.unsqueeze(1)
                
                # Dead code reset - use float32 to match codebook dtype
                dead = batch_size < self.dead_threshold
                if dead.any() and flat_f32.size(0) > 0:
                    n_dead = dead.sum().item()
                    rand_idx = torch.randint(0, flat_f32.size(0), (n_dead,), device=flat_f32.device)
                    self.codebook.data[dead] = flat_f32[rand_idx]
                    self.ema_cluster_size[dead] = 1
                    self.ema_embed_sum[dead] = flat_f32[rand_idx]
        
        commitment_loss = F.mse_loss(z_e_perm, z_q_perm.detach())
        vq_loss = self.commitment_cost * commitment_loss
        
        z_q_st = z_e_perm + (z_q_perm - z_e_perm).detach()
        z_q = z_q_st.permute(0, 4, 1, 2, 3).contiguous()
        
        return z_q, vq_loss, indices.view(z_e_perm.shape[:-1])


class DiagnosticVQVAE(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dims, latent_dim, num_codes,
                 commitment_cost, ema_decay, dead_threshold, pretrained_emb):
        super().__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.latent_dim = latent_dim
        
        # Frozen embeddings
        self.block_emb = nn.Embedding(vocab_size, emb_dim)
        self.block_emb.weight.data.copy_(torch.from_numpy(pretrained_emb))
        self.block_emb.weight.requires_grad = False
        
        # Encoder
        enc = []
        in_ch = emb_dim
        for h in hidden_dims:
            enc.extend([
                nn.Conv3d(in_ch, h, 4, stride=2, padding=1),
                nn.BatchNorm3d(h),
                nn.ReLU(inplace=True),
                ResidualBlock3D(h),
            ])
            in_ch = h
        enc.append(nn.Conv3d(in_ch, latent_dim, 3, padding=1))
        self.encoder = nn.Sequential(*enc)
        
        # Quantizer
        self.quantizer = VectorQuantizerEMA(num_codes, latent_dim, commitment_cost, ema_decay, dead_threshold)
        
        # Decoder
        dec = []
        in_ch = latent_dim
        for h in reversed(hidden_dims):
            dec.extend([
                ResidualBlock3D(in_ch),
                nn.ConvTranspose3d(in_ch, h, 4, stride=2, padding=1),
                nn.BatchNorm3d(h),
                nn.ReLU(inplace=True),
            ])
            in_ch = h
        dec.append(nn.Conv3d(in_ch, vocab_size, 3, padding=1))
        self.decoder = nn.Sequential(*dec)
    
    def forward(self, block_ids):
        x = self.block_emb(block_ids).permute(0, 4, 1, 2, 3).contiguous()
        z_e = self.encoder(x)
        z_q, vq_loss, indices = self.quantizer(z_e)
        logits = self.decoder(z_q)
        return {'logits': logits, 'vq_loss': vq_loss, 'indices': indices, 'z_e': z_e}
    
    def compute_loss_with_diagnostics(self, block_ids, air_tokens, structure_weight):
        out = self(block_ids)
        
        logits = out['logits'].permute(0, 2, 3, 4, 1).contiguous()
        logits_flat = logits.view(-1, self.vocab_size)
        targets_flat = block_ids.view(-1)
        
        # Per-element CE
        ce = F.cross_entropy(logits_flat, targets_flat, reduction='none')
        
        # Structure weighting
        air_dev = air_tokens.to(targets_flat.device)
        is_air = torch.isin(targets_flat, air_dev)
        is_struct = ~is_air
        
        weights = torch.ones_like(ce)
        weights[is_struct] = structure_weight
        recon_loss = (weights * ce).sum() / weights.sum()
        
        total_loss = recon_loss + out['vq_loss']
        
        # Detailed metrics
        with torch.no_grad():
            preds_flat = logits_flat.argmax(-1)
            correct = (preds_flat == targets_flat).float()
            
            acc = correct.mean()
            air_acc = correct[is_air].mean() if is_air.any() else torch.tensor(0.0)
            struct_acc = correct[is_struct].mean() if is_struct.any() else torch.tensor(0.0)
            air_pct = is_air.float().mean()
        
        return {
            'loss': total_loss,
            'recon_loss': recon_loss,
            'vq_loss': out['vq_loss'],
            'accuracy': acc,
            'air_accuracy': air_acc,
            'struct_accuracy': struct_acc,
            'air_pct': air_pct,
            'predictions': preds_flat,
            'targets': targets_flat,
            'is_structure': is_struct,
            'indices': out['indices'],
        }

print("DiagnosticVQVAE defined!")

In [ ]:
# ============================================================
# CELL 7: Diagnostic Tracking Classes
# ============================================================

class PerBlockTracker:
    """Track accuracy per block type."""
    def __init__(self, vocab_size):
        self.correct = np.zeros(vocab_size)
        self.total = np.zeros(vocab_size)
    
    def update(self, preds, targets):
        preds_np = preds.cpu().numpy()
        targets_np = targets.cpu().numpy()
        for t in np.unique(targets_np):
            mask = targets_np == t
            self.total[t] += mask.sum()
            self.correct[t] += (preds_np[mask] == t).sum()
    
    def get_accuracy(self, min_count=10):
        """Return dict of token -> accuracy for tokens with min_count samples."""
        accs = {}
        for t in range(len(self.total)):
            if self.total[t] >= min_count:
                accs[t] = self.correct[t] / self.total[t]
        return accs
    
    def get_top_bottom(self, tok2block, k=20, min_count=10):
        """Return top-k and bottom-k blocks by accuracy."""
        accs = self.get_accuracy(min_count)
        sorted_toks = sorted(accs.keys(), key=lambda t: accs[t], reverse=True)
        
        top = [(tok2block.get(t, f'tok_{t}'), accs[t], int(self.total[t])) for t in sorted_toks[:k]]
        bottom = [(tok2block.get(t, f'tok_{t}'), accs[t], int(self.total[t])) for t in sorted_toks[-k:]]
        return top, bottom


class ConfusionTracker:
    """Track what blocks get confused with what (for structure blocks only)."""
    def __init__(self, max_track=1000):
        self.confusions = Counter()  # (true, pred) -> count
        self.max_track = max_track
    
    def update(self, preds, targets, is_structure):
        preds_np = preds.cpu().numpy()
        targets_np = targets.cpu().numpy()
        is_struct_np = is_structure.cpu().numpy()
        
        # Only track structure block confusions
        wrong = (preds_np != targets_np) & is_struct_np
        for t, p in zip(targets_np[wrong], preds_np[wrong]):
            self.confusions[(int(t), int(p))] += 1
    
    def get_top_confusions(self, tok2block, k=20):
        """Return top-k most common confusions."""
        top = self.confusions.most_common(k)
        result = []
        for (t, p), count in top:
            true_name = tok2block.get(t, f'tok_{t}')
            pred_name = tok2block.get(p, f'tok_{p}')
            result.append((true_name, pred_name, count))
        return result


class GradientTracker:
    """Track gradient norms for encoder vs decoder."""
    def __init__(self):
        self.encoder_norms = []
        self.decoder_norms = []
    
    def compute_norms(self, model):
        enc_norm = 0.0
        dec_norm = 0.0
        
        for name, param in model.named_parameters():
            if param.grad is not None:
                norm = param.grad.norm().item()
                if 'encoder' in name:
                    enc_norm += norm ** 2
                elif 'decoder' in name:
                    dec_norm += norm ** 2
        
        self.encoder_norms.append(enc_norm ** 0.5)
        self.decoder_norms.append(dec_norm ** 0.5)


print("Diagnostic trackers defined!")

In [ ]:
# ============================================================
# CELL 8: Training Functions with Diagnostics
# ============================================================

def train_epoch_diagnostic(model, loader, optimizer, scaler, device, air_tokens, 
                           structure_weight, grad_tracker=None):
    model.train()
    model.quantizer.reset_epoch_stats()
    
    metrics = {'loss': 0, 'recon': 0, 'vq': 0, 'acc': 0, 'air_acc': 0, 'struct_acc': 0, 'air_pct': 0}
    n = 0
    
    optimizer.zero_grad()
    
    for batch_idx, (batch, _) in enumerate(tqdm(loader, desc="Train", leave=False)):
        batch = batch.to(device)
        
        with torch.amp.autocast('cuda', enabled=USE_AMP):
            out = model.compute_loss_with_diagnostics(batch, air_tokens, structure_weight)
            loss = out['loss'] / GRAD_ACCUM_STEPS
        
        scaler.scale(loss).backward()
        
        if (batch_idx + 1) % GRAD_ACCUM_STEPS == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            if grad_tracker:
                grad_tracker.compute_norms(model)
            
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        metrics['loss'] += out['loss'].item()
        metrics['recon'] += out['recon_loss'].item()
        metrics['vq'] += out['vq_loss'].item()
        metrics['acc'] += out['accuracy'].item()
        metrics['air_acc'] += out['air_accuracy'].item()
        metrics['struct_acc'] += out['struct_accuracy'].item()
        metrics['air_pct'] += out['air_pct'].item()
        n += 1
    
    if len(loader) % GRAD_ACCUM_STEPS != 0:
        scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()
    
    metrics['codebook_usage'] = model.quantizer.get_usage_fraction()
    metrics['perplexity'] = model.quantizer.get_perplexity()
    metrics['entropy'] = model.quantizer.get_entropy()
    
    return {k: v/n if k not in ['codebook_usage', 'perplexity', 'entropy'] else v for k, v in metrics.items()}


@torch.no_grad()
def validate_diagnostic(model, loader, device, air_tokens, structure_weight,
                        block_tracker=None, confusion_tracker=None):
    model.eval()
    model.quantizer.reset_epoch_stats()
    
    metrics = {'loss': 0, 'recon': 0, 'acc': 0, 'air_acc': 0, 'struct_acc': 0, 'air_pct': 0}
    n = 0
    
    for batch, _ in tqdm(loader, desc="Val", leave=False):
        batch = batch.to(device)
        
        with torch.amp.autocast('cuda', enabled=USE_AMP):
            out = model.compute_loss_with_diagnostics(batch, air_tokens, structure_weight)
        
        metrics['loss'] += out['loss'].item()
        metrics['recon'] += out['recon_loss'].item()
        metrics['acc'] += out['accuracy'].item()
        metrics['air_acc'] += out['air_accuracy'].item()
        metrics['struct_acc'] += out['struct_accuracy'].item()
        metrics['air_pct'] += out['air_pct'].item()
        n += 1
        
        if block_tracker:
            block_tracker.update(out['predictions'], out['targets'])
        if confusion_tracker:
            confusion_tracker.update(out['predictions'], out['targets'], out['is_structure'])
    
    metrics['codebook_usage'] = model.quantizer.get_usage_fraction()
    metrics['perplexity'] = model.quantizer.get_perplexity()
    metrics['entropy'] = model.quantizer.get_entropy()
    
    return {k: v/n if k not in ['codebook_usage', 'perplexity', 'entropy'] else v for k, v in metrics.items()}


print("Diagnostic training functions defined!")

In [ ]:
# ============================================================
# CELL 9: Reconstruction Visualization
# ============================================================

@torch.no_grad()
def get_reconstruction_samples(model, dataset, device, air_tokens, n_samples=5):
    """Get original and reconstructed structures for visualization."""
    model.eval()
    samples = []
    
    indices = np.random.choice(len(dataset), n_samples, replace=False)
    
    for idx in indices:
        original, _ = dataset[idx]
        original = original.unsqueeze(0).to(device)
        
        out = model(original)
        recon = out['logits'].argmax(dim=1).squeeze(0)
        
        # Compute per-structure accuracy
        orig_flat = original.view(-1)
        recon_flat = recon.view(-1)
        
        air_dev = air_tokens.to(device)
        is_struct = ~torch.isin(orig_flat, air_dev)
        
        overall_acc = (orig_flat == recon_flat).float().mean().item()
        struct_acc = (orig_flat[is_struct] == recon_flat[is_struct]).float().mean().item() if is_struct.any() else 0
        
        samples.append({
            'original': original.squeeze(0).cpu().numpy(),
            'reconstructed': recon.cpu().numpy(),
            'overall_acc': overall_acc,
            'struct_acc': struct_acc,
            'idx': idx,
        })
    
    return samples


def visualize_slice(original, reconstructed, z_slice, tok2block, air_tokens, ax_orig, ax_recon):
    """Visualize a single z-slice of original vs reconstructed."""
    orig_slice = original[:, :, z_slice]
    recon_slice = reconstructed[:, :, z_slice]
    
    # Create color maps
    def get_color(tok):
        if tok in air_tokens:
            return [1, 1, 1]  # White for air
        # Hash-based coloring
        np.random.seed(tok)
        return np.random.rand(3)
    
    orig_rgb = np.zeros((*orig_slice.shape, 3))
    recon_rgb = np.zeros((*recon_slice.shape, 3))
    
    for i in range(orig_slice.shape[0]):
        for j in range(orig_slice.shape[1]):
            orig_rgb[i, j] = get_color(orig_slice[i, j])
            recon_rgb[i, j] = get_color(recon_slice[i, j])
    
    ax_orig.imshow(orig_rgb)
    ax_orig.set_title('Original')
    ax_orig.axis('off')
    
    ax_recon.imshow(recon_rgb)
    ax_recon.set_title('Reconstructed')
    ax_recon.axis('off')


print("Visualization functions defined!")

In [ ]:
# ============================================================
# CELL 10: Category Accuracy Analysis
# ============================================================

def compute_category_accuracy(block_tracker, categories, air_tokens):
    """Compute accuracy per block category."""
    per_block = block_tracker.get_accuracy(min_count=5)
    
    cat_acc = {}
    for cat, toks in categories.items():
        valid_toks = [t for t in toks if t in per_block and t not in air_tokens]
        if valid_toks:
            cat_acc[cat] = np.mean([per_block[t] for t in valid_toks])
    
    return cat_acc


print("Category analysis defined!")

In [ ]:
# ============================================================
# CELL 11: Main Experiment Runner
# ============================================================

def run_full_experiment(name, embeddings, emb_dim, seed, air_tokens, train_ds, val_ds):
    """Run one full experiment with all diagnostics."""
    print(f"\n{'='*70}")
    print(f"{name} (seed={seed}, dim={emb_dim})")
    print(f"{'='*70}")
    
    # Set seeds
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.empty_cache()
    
    # Data loaders (re-create with seed)
    g = torch.Generator()
    g.manual_seed(seed)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True, generator=g)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)
    
    # Create model
    model = DiagnosticVQVAE(
        vocab_size=VOCAB_SIZE,
        emb_dim=emb_dim,
        hidden_dims=HIDDEN_DIMS,
        latent_dim=LATENT_DIM,
        num_codes=NUM_CODEBOOK_ENTRIES,
        commitment_cost=COMMITMENT_COST,
        ema_decay=EMA_DECAY,
        dead_threshold=DEAD_CODE_THRESHOLD,
        pretrained_emb=embeddings,
    ).to(device)
    
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable params: {trainable:,}")
    
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
    scaler = torch.amp.GradScaler('cuda', enabled=USE_AMP)
    
    # Trackers
    grad_tracker = GradientTracker()
    
    history = {
        'train_loss': [], 'train_recon': [], 'train_vq': [],
        'train_acc': [], 'train_air_acc': [], 'train_struct_acc': [],
        'train_codebook_usage': [], 'train_perplexity': [], 'train_entropy': [],
        'val_loss': [], 'val_acc': [], 'val_air_acc': [], 'val_struct_acc': [],
        'val_codebook_usage': [], 'val_perplexity': [], 'val_entropy': [],
        'encoder_grad_norm': [], 'decoder_grad_norm': [],
    }
    
    best_struct_acc = 0
    start = time.time()
    
    for epoch in range(EPOCHS):
        train_m = train_epoch_diagnostic(model, train_loader, optimizer, scaler, device,
                                         air_tokens, STRUCTURE_WEIGHT, grad_tracker)
        val_m = validate_diagnostic(model, val_loader, device, air_tokens, STRUCTURE_WEIGHT)
        
        # Record
        history['train_loss'].append(train_m['loss'])
        history['train_recon'].append(train_m['recon'])
        history['train_vq'].append(train_m['vq'])
        history['train_acc'].append(train_m['acc'])
        history['train_air_acc'].append(train_m['air_acc'])
        history['train_struct_acc'].append(train_m['struct_acc'])
        history['train_codebook_usage'].append(train_m['codebook_usage'])
        history['train_perplexity'].append(train_m['perplexity'])
        history['train_entropy'].append(train_m['entropy'])
        history['val_loss'].append(val_m['loss'])
        history['val_acc'].append(val_m['acc'])
        history['val_air_acc'].append(val_m['air_acc'])
        history['val_struct_acc'].append(val_m['struct_acc'])
        history['val_codebook_usage'].append(val_m['codebook_usage'])
        history['val_perplexity'].append(val_m['perplexity'])
        history['val_entropy'].append(val_m['entropy'])
        
        # Gradient norms (average over accumulation steps)
        if grad_tracker.encoder_norms:
            history['encoder_grad_norm'].append(np.mean(grad_tracker.encoder_norms[-100:]))
            history['decoder_grad_norm'].append(np.mean(grad_tracker.decoder_norms[-100:]))
        
        if val_m['struct_acc'] > best_struct_acc:
            best_struct_acc = val_m['struct_acc']
        
        print(f"Epoch {epoch+1:2d} | Struct: {train_m['struct_acc']:.1%} | "
              f"Val: {val_m['struct_acc']:.1%} | Perp: {train_m['perplexity']:.0f} | "
              f"CB: {train_m['codebook_usage']:.1%}")
    
    train_time = time.time() - start
    
    # Final detailed validation
    print("\nFinal detailed validation...")
    block_tracker = PerBlockTracker(VOCAB_SIZE)
    confusion_tracker = ConfusionTracker()
    
    final_val = validate_diagnostic(model, val_loader, device, air_tokens, STRUCTURE_WEIGHT,
                                    block_tracker, confusion_tracker)
    
    # Get per-block and confusion analysis
    top_blocks, bottom_blocks = block_tracker.get_top_bottom(tok2block, k=TOP_K_BLOCKS)
    top_confusions = confusion_tracker.get_top_confusions(tok2block, k=20)
    category_acc = compute_category_accuracy(block_tracker, BLOCK_CATEGORIES, AIR_TOKENS)
    
    # Get reconstruction samples
    samples = get_reconstruction_samples(model, val_ds, device, air_tokens, NUM_RECONSTRUCTION_SAMPLES)
    
    # Compile results
    results = {
        'name': name,
        'seed': seed,
        'emb_dim': emb_dim,
        'best_struct_acc': best_struct_acc,
        'final_struct_acc': final_val['struct_acc'],
        'final_val_loss': final_val['loss'],
        'final_perplexity': final_val['perplexity'],
        'final_codebook_usage': final_val['codebook_usage'],
        'training_time': train_time,
        'history': history,
        'top_blocks': top_blocks,
        'bottom_blocks': bottom_blocks,
        'top_confusions': top_confusions,
        'category_accuracy': category_acc,
        'reconstruction_samples': samples,
    }
    
    del model, optimizer, scaler
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return results


print("Main experiment runner defined!")

In [ ]:
# ============================================================
# CELL 12: Run All Experiments
# ============================================================

print("="*70)
print("VQ-VAE EMBEDDING VALIDATION V3 - COMPREHENSIVE DIAGNOSTICS")
print("="*70)
print(f"Seeds: {BASE_SEEDS}")
print(f"Epochs: {EPOCHS}")
print(f"Embeddings: V1, V3, Random (V2 dropped)")
print()

all_results = {}

# Run each embedding with multiple seeds
for emb_name, emb_data in EMBEDDINGS_BASE.items():
    all_results[emb_name] = []
    for seed in BASE_SEEDS:
        result = run_full_experiment(
            name=emb_name,
            embeddings=emb_data['embeddings'],
            emb_dim=emb_data['dim'],
            seed=seed,
            air_tokens=AIR_TOKENS_TENSOR,
            train_ds=train_dataset,
            val_ds=val_dataset,
        )
        all_results[emb_name].append(result)

# Random embeddings (fresh random per seed)
all_results['Random'] = []
for seed in BASE_SEEDS:
    np.random.seed(seed)
    rand_emb = np.random.randn(VOCAB_SIZE, 32).astype(np.float32) * v1_std
    
    result = run_full_experiment(
        name='Random',
        embeddings=rand_emb,
        emb_dim=32,
        seed=seed,
        air_tokens=AIR_TOKENS_TENSOR,
        train_ds=train_dataset,
        val_ds=val_dataset,
    )
    all_results['Random'].append(result)

print("\n" + "="*70)
print("ALL EXPERIMENTS COMPLETE")
print("="*70)

In [ ]:
# ============================================================
# CELL 13: Statistical Analysis
# ============================================================

print("\n" + "="*70)
print("STATISTICAL ANALYSIS")
print("="*70)

# Compute mean and std for each embedding
summary_stats = {}
for name, runs in all_results.items():
    struct_accs = [r['best_struct_acc'] for r in runs]
    perplexities = [r['final_perplexity'] for r in runs]
    cb_usages = [r['final_codebook_usage'] for r in runs]
    
    summary_stats[name] = {
        'struct_acc_mean': np.mean(struct_accs),
        'struct_acc_std': np.std(struct_accs),
        'struct_acc_all': struct_accs,
        'perplexity_mean': np.mean(perplexities),
        'perplexity_std': np.std(perplexities),
        'cb_usage_mean': np.mean(cb_usages),
        'cb_usage_std': np.std(cb_usages),
    }

# Print results table
print("\n{:<10} {:>18} {:>18} {:>18}".format(
    "Embedding", "Struct Acc", "Perplexity", "CB Usage"
))
print("-"*70)

sorted_names = sorted(summary_stats.keys(), key=lambda x: summary_stats[x]['struct_acc_mean'], reverse=True)

for name in sorted_names:
    s = summary_stats[name]
    if NUM_SEEDS > 1:
        print("{:<10} {:>8.1%} +/- {:>5.1%}   {:>6.0f} +/- {:>5.0f}   {:>6.1%} +/- {:>4.1%}".format(
            name,
            s['struct_acc_mean'], s['struct_acc_std'],
            s['perplexity_mean'], s['perplexity_std'],
            s['cb_usage_mean'], s['cb_usage_std'],
        ))
    else:
        print("{:<10} {:>15.1%}   {:>12.0f}   {:>12.1%}".format(
            name,
            s['struct_acc_mean'],
            s['perplexity_mean'],
            s['cb_usage_mean'],
        ))

# Statistical significance tests (only if multiple seeds)
if NUM_SEEDS > 1:
    print("\n" + "="*70)
    print("STATISTICAL SIGNIFICANCE (t-tests)")
    print("="*70)

    names = ['V1', 'V3', 'Random']
    for i, n1 in enumerate(names):
        for n2 in names[i+1:]:
            acc1 = summary_stats[n1]['struct_acc_all']
            acc2 = summary_stats[n2]['struct_acc_all']
            
            t_stat, p_val = stats.ttest_ind(acc1, acc2)
            
            sig = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
            
            diff = summary_stats[n1]['struct_acc_mean'] - summary_stats[n2]['struct_acc_mean']
            print(f"{n1} vs {n2}: diff={diff:+.1%}, p={p_val:.3f} {sig}")
else:
    print("\n(Statistical significance tests skipped - need multiple seeds)")

In [ ]:
# ============================================================
# CELL 14: Per-Block Analysis
# ============================================================

print("\n" + "="*70)
print("PER-BLOCK ANALYSIS (from best run of each embedding)")
print("="*70)

# Get best run for each embedding
best_runs = {}
for name, runs in all_results.items():
    best_idx = np.argmax([r['best_struct_acc'] for r in runs])
    best_runs[name] = runs[best_idx]

# Top and bottom blocks for winner
winner = sorted_names[0]
print(f"\n=== {winner} (Best Embedding) ===")
print(f"\nTop {TOP_K_BLOCKS} easiest blocks to reconstruct:")
for block, acc, count in best_runs[winner]['top_blocks']:
    print(f"  {acc:>6.1%} ({count:>5} samples): {block}")

print(f"\nBottom {TOP_K_BLOCKS} hardest blocks to reconstruct:")
for block, acc, count in best_runs[winner]['bottom_blocks']:
    print(f"  {acc:>6.1%} ({count:>5} samples): {block}")

In [ ]:
# ============================================================
# CELL 15: Confusion Analysis
# ============================================================

print("\n" + "="*70)
print("TOP CONFUSIONS (What gets mistaken for what?)")
print("="*70)

for name in ['V1', 'V3', 'Random']:
    print(f"\n=== {name} ===")
    for true_block, pred_block, count in best_runs[name]['top_confusions'][:10]:
        print(f"  {count:>5}x: {true_block[:30]:<30} -> {pred_block[:30]}")

In [ ]:
# ============================================================
# CELL 16: Category Accuracy Comparison
# ============================================================

print("\n" + "="*70)
print("ACCURACY BY BLOCK CATEGORY")
print("="*70)

# Gather category accuracies
cat_data = {}
for name in ['V1', 'V3', 'Random']:
    cat_data[name] = best_runs[name]['category_accuracy']

# Print table
all_cats = sorted(set().union(*[set(d.keys()) for d in cat_data.values()]))

print("\n{:<15} {:>10} {:>10} {:>10}".format("Category", "V1", "V3", "Random"))
print("-"*50)

for cat in all_cats:
    row = [cat]
    for name in ['V1', 'V3', 'Random']:
        if cat in cat_data[name]:
            row.append(f"{cat_data[name][cat]:.1%}")
        else:
            row.append("-")
    print("{:<15} {:>10} {:>10} {:>10}".format(*row))

# Plot category comparison
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(all_cats))
width = 0.25
colors = {'V1': 'blue', 'V3': 'purple', 'Random': 'red'}

for i, name in enumerate(['V1', 'V3', 'Random']):
    vals = [cat_data[name].get(cat, 0) for cat in all_cats]
    ax.bar(x + i*width, vals, width, label=name, color=colors[name], alpha=0.8)

ax.set_ylabel('Accuracy')
ax.set_title('Reconstruction Accuracy by Block Category')
ax.set_xticks(x + width)
ax.set_xticklabels(all_cats, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/category_accuracy.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# CELL 17: Training Dynamics Plots
# ============================================================

fig, axes = plt.subplots(3, 4, figsize=(20, 12))

colors = {'V1': 'blue', 'V3': 'purple', 'Random': 'red'}
names = ['V1', 'V3', 'Random']

# Use first seed for clarity
run_idx = 0

# Row 1: Loss components
ax = axes[0, 0]
for name in names:
    ax.plot(all_results[name][run_idx]['history']['train_loss'], label=name, color=colors[name])
ax.set_title('Total Loss')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[0, 1]
for name in names:
    ax.plot(all_results[name][run_idx]['history']['train_recon'], label=name, color=colors[name])
ax.set_title('Reconstruction Loss')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[0, 2]
for name in names:
    ax.plot(all_results[name][run_idx]['history']['train_vq'], label=name, color=colors[name])
ax.set_title('VQ Loss (Commitment)')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[0, 3]
for name in names:
    ax.plot(all_results[name][run_idx]['history']['val_struct_acc'], label=name, color=colors[name], linewidth=2)
ax.set_title('STRUCTURE ACCURACY (KEY)', fontweight='bold')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

# Row 2: Codebook health
ax = axes[1, 0]
for name in names:
    ax.plot(all_results[name][run_idx]['history']['train_codebook_usage'], label=name, color=colors[name])
ax.axhline(y=0.3, color='gray', linestyle='--', alpha=0.5)
ax.set_title('Codebook Usage %')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1, 1]
for name in names:
    ax.plot(all_results[name][run_idx]['history']['train_perplexity'], label=name, color=colors[name])
ax.set_title('Codebook Perplexity (effective # codes)')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1, 2]
for name in names:
    ax.plot(all_results[name][run_idx]['history']['train_entropy'], label=name, color=colors[name])
ax.set_title('Code Distribution Entropy')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

# Convergence speed
ax = axes[1, 3]
for name in names:
    # Normalize to show relative convergence
    hist = all_results[name][run_idx]['history']['val_struct_acc']
    final = hist[-1]
    normalized = [h/final if final > 0 else 0 for h in hist]
    ax.plot(normalized, label=name, color=colors[name])
ax.set_title('Convergence Speed (normalized)')
ax.set_xlabel('Epoch')
ax.set_ylabel('% of final accuracy')
ax.legend()
ax.grid(True, alpha=0.3)

# Row 3: Gradient analysis and train/val gap
ax = axes[2, 0]
for name in names:
    if all_results[name][run_idx]['history']['encoder_grad_norm']:
        ax.plot(all_results[name][run_idx]['history']['encoder_grad_norm'], label=name, color=colors[name])
ax.set_title('Encoder Gradient Norm')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[2, 1]
for name in names:
    if all_results[name][run_idx]['history']['decoder_grad_norm']:
        ax.plot(all_results[name][run_idx]['history']['decoder_grad_norm'], label=name, color=colors[name])
ax.set_title('Decoder Gradient Norm')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

# Train/val gap (overfitting indicator)
ax = axes[2, 2]
for name in names:
    train = all_results[name][run_idx]['history']['train_struct_acc']
    val = all_results[name][run_idx]['history']['val_struct_acc']
    gap = [t - v for t, v in zip(train, val)]
    ax.plot(gap, label=name, color=colors[name])
ax.axhline(y=0, color='gray', linestyle='--')
ax.set_title('Train-Val Gap (overfitting indicator)')
ax.set_xlabel('Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

# Final bar chart with error bars
ax = axes[2, 3]
means = [summary_stats[name]['struct_acc_mean'] for name in names]
stds = [summary_stats[name]['struct_acc_std'] for name in names]
bar_colors = [colors[name] for name in names]

bars = ax.bar(names, means, yerr=stds, capsize=5, color=bar_colors, edgecolor='black')
ax.set_title('Final Structure Accuracy (mean +/- std)')
ax.set_ylabel('Accuracy')

for bar, mean in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{mean:.1%}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/training_dynamics.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# CELL 18: Reconstruction Visualization
# ============================================================

print("\n" + "="*70)
print("RECONSTRUCTION SAMPLES")
print("="*70)

# Visualize reconstructions from best embedding
winner = sorted_names[0]
samples = best_runs[winner]['reconstruction_samples']

fig, axes = plt.subplots(len(samples), 6, figsize=(18, 3*len(samples)))

for row, sample in enumerate(samples):
    orig = sample['original']
    recon = sample['reconstructed']
    
    # Show 3 z-slices
    for col, z in enumerate([8, 16, 24]):
        if z < orig.shape[2]:
            visualize_slice(orig, recon, z, tok2block, AIR_TOKENS,
                           axes[row, col*2], axes[row, col*2+1])
    
    axes[row, 0].set_ylabel(f"Sample {row+1}\nStruct: {sample['struct_acc']:.1%}", fontsize=10)

plt.suptitle(f'Reconstruction Samples ({winner})', fontsize=14)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/reconstruction_samples.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# CELL 19: Save All Results
# ============================================================

# Save summary statistics
summary_output = {}
for emb_name, emb_stats in summary_stats.items():  # Renamed to avoid shadowing scipy.stats
    summary_output[emb_name] = {
        'struct_acc_mean': float(emb_stats['struct_acc_mean']),
        'struct_acc_std': float(emb_stats['struct_acc_std']),
        'struct_acc_all': [float(x) for x in emb_stats['struct_acc_all']],
        'perplexity_mean': float(emb_stats['perplexity_mean']),
        'perplexity_std': float(emb_stats['perplexity_std']),
        'cb_usage_mean': float(emb_stats['cb_usage_mean']),
        'cb_usage_std': float(emb_stats['cb_usage_std']),
    }

with open(f"{OUTPUT_DIR}/validation_v3_summary.json", 'w') as f:
    json.dump(summary_output, f, indent=2)

# Save detailed results (without reconstruction samples - too large)
detailed_output = {}
for emb_name, runs in all_results.items():
    detailed_output[emb_name] = []
    for run in runs:
        run_data = {
            'seed': run['seed'],
            'best_struct_acc': float(run['best_struct_acc']),
            'final_struct_acc': float(run['final_struct_acc']),
            'final_perplexity': float(run['final_perplexity']),
            'final_codebook_usage': float(run['final_codebook_usage']),
            'training_time': float(run['training_time']),
            'history': {k: [float(x) for x in v] for k, v in run['history'].items()},
            'top_blocks': run['top_blocks'],
            'bottom_blocks': run['bottom_blocks'],
            'top_confusions': run['top_confusions'],
            'category_accuracy': {k: float(v) for k, v in run['category_accuracy'].items()},
        }
        detailed_output[emb_name].append(run_data)

with open(f"{OUTPUT_DIR}/validation_v3_detailed.json", 'w') as f:
    json.dump(detailed_output, f, indent=2)

# Save config
config = {
    'seeds': BASE_SEEDS,
    'epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'hidden_dims': HIDDEN_DIMS,
    'latent_dim': LATENT_DIM,
    'num_codes': NUM_CODEBOOK_ENTRIES,
    'commitment_cost': COMMITMENT_COST,
    'ema_decay': EMA_DECAY,
    'structure_weight': STRUCTURE_WEIGHT,
    'air_tokens': AIR_TOKENS_LIST,
}

with open(f"{OUTPUT_DIR}/validation_v3_config.json", 'w') as f:
    json.dump(config, f, indent=2)

print("Results saved:")
print(f"  - {OUTPUT_DIR}/validation_v3_summary.json")
print(f"  - {OUTPUT_DIR}/validation_v3_detailed.json")
print(f"  - {OUTPUT_DIR}/validation_v3_config.json")
print(f"  - {OUTPUT_DIR}/category_accuracy.png")
print(f"  - {OUTPUT_DIR}/training_dynamics.png")
print(f"  - {OUTPUT_DIR}/reconstruction_samples.png")

In [ ]:
# ============================================================
# CELL 20: Final Conclusions
# ============================================================

print("\n" + "="*70)
print("FINAL CONCLUSIONS")
print("="*70)

print("\n1. RANKING (by structure accuracy):")
for i, name in enumerate(sorted_names, 1):
    s = summary_stats[name]
    if NUM_SEEDS > 1:
        print(f"   {i}. {name}: {s['struct_acc_mean']:.1%} +/- {s['struct_acc_std']:.1%}")
    else:
        print(f"   {i}. {name}: {s['struct_acc_mean']:.1%}")

winner = sorted_names[0]
random_mean = summary_stats['Random']['struct_acc_mean']
winner_mean = summary_stats[winner]['struct_acc_mean']
improvement = (winner_mean - random_mean) / random_mean * 100

print(f"\n2. WINNER: {winner}")
print(f"   Improvement over random: {improvement:+.1f}%")

if NUM_SEEDS > 1:
    print("\n3. STATISTICAL SIGNIFICANCE:")
    v1_accs = summary_stats['V1']['struct_acc_all']
    v3_accs = summary_stats['V3']['struct_acc_all']
    rand_accs = summary_stats['Random']['struct_acc_all']

    _, p_v1_random = stats.ttest_ind(v1_accs, rand_accs)
    _, p_v1_v3 = stats.ttest_ind(v1_accs, v3_accs)
    _, p_v3_random = stats.ttest_ind(v3_accs, rand_accs)

    print(f"   V1 vs Random: p={p_v1_random:.4f} {'(significant)' if p_v1_random < 0.05 else '(not significant)'}")
    print(f"   V3 vs Random: p={p_v3_random:.4f} {'(significant)' if p_v3_random < 0.05 else '(not significant)'}")
    print(f"   V1 vs V3: p={p_v1_v3:.4f} {'(significant)' if p_v1_v3 < 0.05 else '(not significant)'}")
else:
    print("\n3. STATISTICAL SIGNIFICANCE:")
    print("   (Skipped - need multiple seeds for t-tests)")

names = ['V1', 'V3', 'Random']

print("\n4. CODEBOOK HEALTH:")
for name in names:
    usage = summary_stats[name]['cb_usage_mean']
    perp = summary_stats[name]['perplexity_mean']
    status = "Healthy" if usage > 0.3 else "WARNING: Possible collapse"
    print(f"   {name}: {usage:.1%} usage, {perp:.0f} effective codes - {status}")

print("\n5. CATEGORY INSIGHTS:")
# Find categories where V1 significantly beats Random
v1_cats = best_runs['V1']['category_accuracy']
rand_cats = best_runs['Random']['category_accuracy']
best_cats = []
for cat in v1_cats:
    if cat in rand_cats:
        diff = v1_cats[cat] - rand_cats[cat]
        if diff > 0.05:
            best_cats.append((cat, diff))

best_cats.sort(key=lambda x: x[1], reverse=True)
print("   Categories where embeddings help most:")
for cat, diff in best_cats[:5]:
    print(f"   - {cat}: +{diff:.1%}")

print("\n6. MAIN FAILURE MODES:")
print("   Most common confusions (from winner):")
for true_b, pred_b, count in best_runs[winner]['top_confusions'][:5]:
    print(f"   - {true_b[:25]} -> {pred_b[:25]} ({count}x)")

print("\n" + "="*70)
print("RECOMMENDATIONS")
print("="*70)

v1_mean = summary_stats['V1']['struct_acc_mean']
v3_mean = summary_stats['V3']['struct_acc_mean']

if v1_mean > v3_mean:
    diff_pct = (v1_mean - v3_mean) / v3_mean * 100
    print(f"\n-> V1 outperformed V3 by {diff_pct:.1f}%")
    if diff_pct > 5:
        print("   Recommendation: Use V1 embeddings for VQ-VAE")
    else:
        print("   Difference is small. Consider V3 for interpretability benefits.")
else:
    diff_pct = (v3_mean - v1_mean) / v1_mean * 100
    print(f"\n-> V3 outperformed V1 by {diff_pct:.1f}%")
    if diff_pct > 5:
        print("   Recommendation: Use V3 compositional embeddings for VQ-VAE")
    else:
        print("   Difference is small. V3 has interpretability benefits.")

print("\n" + "="*70)